# MFEGSN — Pipeline Colab (Marker + LangExtract)

Notebook optimisé pour une exécution **pas à pas** sur Google Colab.

**🚀 Optimisation:** Les modèles (~4GB) sont sauvegardés sur Google Drive après le premier téléchargement.
Les sessions suivantes chargeront les modèles depuis Drive en quelques secondes.

**Ordre recommandé:** Étapes 1 → 12

## Étape 1 — Monter Google Drive
**IMPORTANT:** Exécutez cette cellule EN PREMIER pour permettre le cache des modèles.

In [1]:
from google.colab import drive
drive.mount("/content/drive")

# Créer le dossier de cache sur Drive
import os
from pathlib import Path

# === CONFIGURATION DU CACHE ===
DRIVE_CACHE_DIR = Path("/content/drive/MyDrive/.mfegsn_cache")
DRIVE_CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Sous-dossiers pour chaque type de cache
HF_CACHE_DRIVE = DRIVE_CACHE_DIR / "huggingface"
TORCH_CACHE_DRIVE = DRIVE_CACHE_DIR / "torch"
DATALAB_CACHE_DRIVE = DRIVE_CACHE_DIR / "datalab"

for cache_dir in [HF_CACHE_DRIVE, TORCH_CACHE_DRIVE, DATALAB_CACHE_DRIVE]:
    cache_dir.mkdir(parents=True, exist_ok=True)

# Configurer les variables d'environnement pour utiliser le cache Drive
os. environ["HF_HOME"] = str(HF_CACHE_DRIVE)
os.environ["TRANSFORMERS_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["HUGGINGFACE_HUB_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["TORCH_HOME"] = str(TORCH_CACHE_DRIVE)
os.environ["XDG_CACHE_HOME"] = str(DRIVE_CACHE_DIR)

# Vérifier la taille du cache existant
def get_dir_size(path):
    total = 0
    if path.exists():
        for f in path.rglob("*"):
            if f.is_file():
                total += f.stat().st_size
    return total / 1e9

cache_size = get_dir_size(DRIVE_CACHE_DIR)

print("="*60)
print("✅ Google Drive monté! ")
print("="*60)
print(f"📂 Cache Drive:  {DRIVE_CACHE_DIR}")
print(f"💾 Taille du cache: {cache_size:.2f} GB")
if cache_size > 3:
    print("🚀 Modèles déjà en cache!  Le chargement sera rapide.")
else:
    print("📥 Premier lancement:  les modèles seront téléchargés et mis en cache.")
print("="*60)

Mounted at /content/drive
✅ Google Drive monté! 
📂 Cache Drive:  /content/drive/MyDrive/.mfegsn_cache
💾 Taille du cache: 3.53 GB
🚀 Modèles déjà en cache!  Le chargement sera rapide.


## Étape 2 — Installer les dépendances
Installation des packages Python nécessaires.

In [ ]:
# Dépendances système
!apt-get update -qq
!apt-get install -y zstd -qq

# Dépendances Python
!python -m pip install -q --upgrade pip
!python -m pip install -q marker-pdf[full] langextract google-generativeai pillow pymupdf

# Installer hf_transfer pour des téléchargements plus rapides (sans casser les dépendances)
!pip install -q hf_transfer --no-deps

# Activer hf_transfer
import os
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# Vérifier les versions
print("\n📦 Versions installées:")
!pip show huggingface_hub 2>/dev/null | grep Version
!pip show transformers 2>/dev/null | grep Version
!pip show marker-pdf 2>/dev/null | grep Version
!pip show pymupdf 2>/dev/null | grep Version

print("\n✅ Dépendances installées.")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package zstd.
(Reading database ... 121689 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...

📦 Versions installées:
Version: 0.36.0
Version: 4.57.6
Version: 1.10.1

✅ Dépendances installées.


## Étape 3 — Configurer les dossiers de travail
Modifiez les chemins selon votre Drive.

In [3]:
from pathlib import Path

# === MODIFIEZ CES CHEMINS ===
INPUT_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF")
OUTPUT_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD")

assert INPUT_DIR.exists(), f"❌ Dossier introuvable: {INPUT_DIR}"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Dossier global pour les logs du pipeline
GLOBAL_LOGS_DIR = OUTPUT_DIR / "_PIPELINE_LOGS"
GLOBAL_LOGS_DIR.mkdir(parents=True, exist_ok=True)

def get_doc_folders(doc_name):
    """Crée et retourne les dossiers pour un document donné."""
    doc_dir = OUTPUT_DIR / doc_name
    doc_dir.mkdir(parents=True, exist_ok=True)

    folders = {
        "root": doc_dir,
        "figures": doc_dir / "_FIGURES",
        "references": doc_dir / "_REFERENCES",
        "analyses": doc_dir / "_ANALYSES",
        "logs": doc_dir / "_LOGS",
    }

    for folder in folders.values():
        folder.mkdir(parents=True, exist_ok=True)

    return folders

pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])
print("="*60)
print("📁 CONFIGURATION")
print("="*60)
print(f"📂 Entrée: {INPUT_DIR}")
print(f"📂 Sortie: {OUTPUT_DIR}")
print(f"📄 PDFs: {len(pdf_files)}")
print(f"📂 Structure par fichier:  <nom_fichier>/")
print(f"   ├── _ANALYSES/")
print(f"   ├── _FIGURES/")
print(f"   ├── _LOGS/")
print(f"   └── _REFERENCES/")
print("="*60)

📁 CONFIGURATION
📂 Entrée: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF
📂 Sortie: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD
📄 PDFs: 54
📂 Structure par fichier:  <nom_fichier>/
   ├── _ANALYSES/
   ├── _FIGURES/
   ├── _LOGS/
   └── _REFERENCES/


## Étape 4 — Renommer les fichiers PDF (Optionnel)
Cette étape permet de renommer en lot les fichiers PDF en remplaçant une chaîne de caractères par une autre.

**Utile pour:** Nettoyer les noms de fichiers, supprimer des préfixes/suffixes indésirables, corriger des caractères

**⚠️ Cette étape est optionnelle.** Mettez `RENAME_FILES = False` pour la sauter.

In [ ]:
# === CONFIGURATION DU RENOMMAGE ===
RENAME_FILES = True  # Mettre à False pour sauter cette étape

# === RÈGLES DE REMPLACEMENT ===
# Ajoutez autant de règles que nécessaire
# Format: ("chaîne à chercher", "chaîne de remplacement")
RENAME_RULES = [
    ("_ocr", ""),           # Exemple: supprimer "_ocr" des noms
    # ("ancien", "nouveau"),  # Ajoutez d'autres règles ici
    # (" - ", "_"),           # Remplacer " - " par "_"
    # ("  ", " "),            # Supprimer les doubles espaces
]

# === OPTIONS AVANCÉES ===
RENAME_PREVIEW_ONLY = True   # True = afficher sans renommer, False = renommer réellement
RENAME_TARGET_DIR = INPUT_DIR  # Dossier cible (INPUT_DIR par défaut)

if RENAME_FILES:
    from pathlib import Path
    import shutil
    
    print("="*60)
    print("📝 RENOMMAGE DES FICHIERS PDF")
    print("="*60)
    
    if RENAME_PREVIEW_ONLY:
        print("⚠️ MODE APERÇU: Aucun fichier ne sera renommé")
        print("   Mettez RENAME_PREVIEW_ONLY = False pour appliquer les changements")
    else:
        print("🔄 MODE ACTIF: Les fichiers seront renommés!")
    
    print(f"\n📂 Dossier: {RENAME_TARGET_DIR}")
    print(f"\n📋 Règles de remplacement:")
    for old, new in RENAME_RULES:
        display_new = f'"{new}"' if new else "(supprimé)"
        print(f'   "{old}" → {display_new}')
    
    # Lister les fichiers PDF
    pdf_files = sorted([p for p in RENAME_TARGET_DIR.iterdir() if p.suffix.lower() == ".pdf"])
    print(f"\n📄 {len(pdf_files)} fichiers PDF trouvés")
    
    # Calculer les nouveaux noms
    rename_plan = []
    skipped = 0
    
    for pdf_path in pdf_files:
        old_name = pdf_path.stem
        new_name = old_name
        
        # Appliquer toutes les règles de remplacement
        for search_str, replace_str in RENAME_RULES:
            if search_str:  # Ignorer les règles vides
                new_name = new_name.replace(search_str, replace_str)
        
        # Nettoyer les espaces multiples et les espaces en début/fin
        new_name = " ".join(new_name.split()).strip()
        
        # Vérifier si le nom a changé
        if new_name != old_name:
            new_path = pdf_path.parent / f"{new_name}.pdf"
            rename_plan.append({
                "old_path": pdf_path,
                "new_path": new_path,
                "old_name": old_name,
                "new_name": new_name,
            })
        else:
            skipped += 1
    
    print(f"\n📊 Résultat de l'analyse:")
    print(f"   🔄 À renommer: {len(rename_plan)}")
    print(f"   ⏭️ Inchangés: {skipped}")
    
    if rename_plan:
        print(f"\n{'='*60}")
        print("📋 APERÇU DES CHANGEMENTS")
        print("="*60)
        
        for idx, item in enumerate(rename_plan, 1):
            print(f"\n{idx}. {item['old_name']}")
            print(f"   → {item['new_name']}")
        
        # Appliquer les changements si pas en mode aperçu
        if not RENAME_PREVIEW_ONLY:
            print(f"\n{'='*60}")
            print("🔄 APPLICATION DES CHANGEMENTS")
            print("="*60)
            
            success_count = 0
            error_count = 0
            
            for item in rename_plan:
                try:
                    # Vérifier si le fichier destination existe déjà
                    if item['new_path'].exists():
                        print(f"⚠️ {item['old_name']} → Fichier destination existe déjà!")
                        error_count += 1
                        continue
                    
                    # Renommer le fichier
                    item['old_path'].rename(item['new_path'])
                    print(f"✅ {item['old_name']} → {item['new_name']}")
                    success_count += 1
                    
                except Exception as e:
                    print(f"❌ {item['old_name']} → Erreur: {str(e)[:50]}")
                    error_count += 1
            
            print(f"\n{'='*60}")
            print("📊 RÉSUMÉ DU RENOMMAGE")
            print("="*60)
            print(f"✅ Renommés: {success_count}")
            print(f"❌ Erreurs: {error_count}")
            print("="*60)
            
            # Mettre à jour la liste des PDFs
            pdf_files = sorted([p for p in RENAME_TARGET_DIR.iterdir() if p.suffix.lower() == ".pdf"])
            print(f"\n📄 Nouveaux fichiers: {len(pdf_files)}")
        else:
            print(f"\n{'='*60}")
            print("💡 Pour appliquer ces changements:")
            print("   1. Mettez RENAME_PREVIEW_ONLY = False")
            print("   2. Réexécutez cette cellule")
            print("="*60)
    else:
        print("\n✅ Aucun fichier à renommer (aucune règle ne s'applique)")
        
else:
    print("ℹ️ Renommage désactivé. Mettez RENAME_FILES = True pour l'activer.")

## Étape 5 — Ollama + Gemma 3 4B
Installation et démarrage de Gemma 3 4B via Ollama.

In [4]:
USE_OLLAMA = True  # Activé par défaut pour Gemma local

ollama_process = None

if USE_OLLAMA:
    import subprocess
    import time
    import os

    # Installer requests si nécessaire
    try:
        import requests
    except ImportError:
        !pip install -q requests
        import requests

    # Installer Ollama
    print("📦 Installation d'Ollama...")
    !curl -fsSL https://ollama.com/install.sh 2>/dev/null | sh 2>&1 | tail -n 3

    # Démarrer Ollama en arrière-plan
    ollama_process = subprocess.Popen(
        ["ollama", "serve"],
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    print("⏳ Démarrage du serveur Ollama...")

    # Attendre que le serveur soit prêt (avec timeout de 30 secondes)
    server_ready = False
    for i in range(30):
        try:
            response = requests.get("http://localhost:11434/api/tags", timeout=2)
            if response.status_code == 200:
                print("✅ Serveur Ollama prêt!")
                server_ready = True
                break
        except:
            pass
        time.sleep(1)

    if not server_ready:
        print("⚠️ Le serveur Ollama n'a pas démarré à temps")
        print("   Essayez de réexécuter cette cellule.")

    # Télécharger Gemma 3 4B (SANS espace dans le nom!)
    print("\n📥 Téléchargement de Gemma 3 4B (≈3GB)...")
    print("   Cela peut prendre plusieurs minutes...")
    !ollama pull gemma3:4b

    # Vérification
    result = subprocess.run(["ollama", "list"], capture_output=True, text=True)
    if "gemma3:4b" in result.stdout or "gemma3" in result.stdout:
        print("\n✅ Gemma 3 4B téléchargé!")

        # Préchauffer le modèle avec une requête test
        print("🔥 Préchauffage du modèle...")
        test_result = subprocess.run(
            ["ollama", "run", "gemma3:4b", "Réponds uniquement: OK"],
            capture_output=True,
            text=True,
            timeout=30
        )

        if test_result.returncode == 0:
            print("✅ Gemma 3 4B prêt et opérationnel!")
            print("\n" + "="*60)
            print("🤖 OLLAMA CONFIGURÉ")
            print("="*60)
            print("Modèle: gemma3:4b")
            print("Serveur: http://localhost:11434")
            print("="*60)
        else:
            print("⚠️ Avertissement: Le test du modèle a échoué")
            print("   Le modèle devrait quand même fonctionner.")
    else:
        print("❌ Erreur: Gemma 3 4B non trouvé")
        print("Modèles disponibles:")
        print(result.stdout if result.stdout else "Aucun modèle")
else:
    print("ℹ️ Ollama désactivé. Mettez USE_OLLAMA = True pour l'activer.")

📦 Installation d'Ollama...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
⏳ Démarrage du serveur Ollama...
✅ Serveur Ollama prêt!

📥 Téléchargement de Gemma 3 4B (≈3GB)...
   Cela peut prendre plusieurs minutes...


✅ Gemma 3 4B téléchargé!
🔥 Préchauffage du modèle...
✅ Gemma 3 4B prêt et opérationnel!

🤖 OLLAMA CONFIGURÉ
Modèle: gemma3:4b
Serveur: http://localhost:11434


## Étape 6 — OCR des PDFs avec OCRmyPDF (Optionnel)
Cette étape permet d'appliquer l'OCR sur tous les PDFs avant de les traiter avec Marker.

**Utile pour:** PDFs scannés, documents sans texte sélectionnable

**Résultat:** Les PDFs originaux sont remplacés par leur version OCRisée dans `ALLPDF`

**⚠️ Cette étape est optionnelle.** Si vos PDFs contiennent déjà du texte sélectionnable, vous pouvez la sauter.

**Note:** Utilise OCRmyPDF + Tesseract (compatible Colab, pas besoin de Docker)

In [ ]:
# === CONFIGURATION OCR ===
USE_OCR = True  # Mettre à False pour sauter cette étape
OCR_LANGUAGES = ["fra", "eng", "ara"]  # Langues OCR (fra=français, eng=anglais, ara=arabe)

if USE_OCR:
    import subprocess
    import os
    import shutil
    import tempfile
    from pathlib import Path
    
    print("="*60)
    print("🔍 OCR DES PDFS AVEC OCRMYPDF + TESSERACT")
    print("="*60)
    
    # Installation des dépendances OCR
    print("\n📦 Installation des outils OCR...")
    
    # Installer Tesseract, unpaper et les langues
    !sudo apt-get update -qq
    !sudo apt-get install -y -qq tesseract-ocr unpaper ghostscript
    
    # Installer les packs de langues Tesseract
    lang_packages = " ".join([f"tesseract-ocr-{lang}" for lang in OCR_LANGUAGES])
    !sudo apt-get install -y -qq {lang_packages}
    
    # Installer OCRmyPDF et ses dépendances
    !pip install -q ocrmypdf
    
    # Vérifier l'installation
    tesseract_check = subprocess.run(["tesseract", "--version"], capture_output=True, text=True)
    if tesseract_check.returncode == 0:
        version_line = tesseract_check.stdout.split('\n')[0]
        print(f"✅ Tesseract installé: {version_line}")
    else:
        print("❌ Erreur d'installation de Tesseract")
    
    # Lister les langues disponibles
    langs_check = subprocess.run(["tesseract", "--list-langs"], capture_output=True, text=True)
    available_langs = langs_check.stdout.strip().split('\n')[1:]  # Skip header
    print(f"📝 Langues disponibles: {', '.join(available_langs)}")
    
    # Vérifier que nos langues sont installées
    missing_langs = [lang for lang in OCR_LANGUAGES if lang not in available_langs]
    if missing_langs:
        print(f"⚠️ Langues manquantes: {missing_langs}")
    
    # Importer ocrmypdf
    import ocrmypdf
    print(f"✅ OCRmyPDF version: {ocrmypdf.__version__}")
    
    # Détecter le nombre de CPU disponibles
    import multiprocessing
    num_cpus = multiprocessing.cpu_count()
    print(f"🖥️ CPUs disponibles: {num_cpus}")
    
    # Configurer le dossier (source = destination)
    PDF_DIR = Path("/content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLPDF")
    
    # Dossier temporaire pour les fichiers OCRisés
    TEMP_OCR_DIR = Path("/content/temp_ocr")
    TEMP_OCR_DIR.mkdir(parents=True, exist_ok=True)
    
    # Langues pour OCRmyPDF (format: fra+eng+ara)
    ocr_lang_str = "+".join(OCR_LANGUAGES)
    
    print(f"\n📁 Dossier PDF: {PDF_DIR}")
    print(f"🔄 Mode: Remplacement des originaux par version OCR")
    print(f"🌍 Langues OCR: {ocr_lang_str}")
    
    # Lister les PDFs
    pdf_files = sorted([p for p in PDF_DIR.iterdir() if p.suffix.lower() == ".pdf"])
    print(f"\n📄 {len(pdf_files)} PDFs à traiter")
    print("="*60)
    
    success_count = 0
    already_ocr_count = 0
    error_count = 0
    
    for idx, pdf_path in enumerate(pdf_files, 1):
        pdf_name = pdf_path.stem
        temp_output = TEMP_OCR_DIR / f"{pdf_name}.pdf"
        
        print(f"🔍 [{idx}/{len(pdf_files)}] {pdf_name}...", end=" ", flush=True)
        
        try:
            # Appliquer OCR avec OCRmyPDF vers fichier temporaire
            result = ocrmypdf.ocr(
                pdf_path,
                temp_output,
                language=ocr_lang_str,
                skip_text=True,  # Ne pas ré-OCR les pages avec du texte
                optimize=0,      # Pas d'optimisation (plus rapide)
                progress_bar=False,
                jobs=num_cpus,   # Utiliser tous les CPUs
                tesseract_timeout=180,  # Timeout par page
                tesseract_non_ocr_timeout=30
            )
            
            if result == ocrmypdf.ExitCode.ok:
                # Remplacer l'original par la version OCR
                shutil.move(str(temp_output), str(pdf_path))
                print("✅ (remplacé)")
                success_count += 1
            elif result == ocrmypdf.ExitCode.already_done_ocr:
                # Déjà OCRisé, supprimer le temp si créé
                if temp_output.exists():
                    temp_output.unlink()
                print("📋 (déjà OCRisé)")
                already_ocr_count += 1
            else:
                print(f"⚠️ (code: {result}, conservé)")
                if temp_output.exists():
                    temp_output.unlink()
                already_ocr_count += 1
                
        except ocrmypdf.exceptions.PriorOcrFoundError:
            # Le PDF a déjà du texte OCR
            if temp_output.exists():
                temp_output.unlink()
            print("📋 (texte existant)")
            already_ocr_count += 1
        except ocrmypdf.exceptions.EncryptedPdfError:
            print("🔒 (PDF protégé)")
            error_count += 1
        except Exception as e:
            error_msg = str(e)[:60]
            print(f"❌ {error_msg}")
            error_count += 1
            # Nettoyer le fichier temp en cas d'erreur
            if temp_output.exists():
                temp_output.unlink()
    
    # Nettoyer le dossier temporaire
    shutil.rmtree(TEMP_OCR_DIR, ignore_errors=True)
    
    print("\n" + "="*60)
    print("📊 RÉSUMÉ OCR")
    print("="*60)
    print(f"✅ OCR appliqué et remplacé: {success_count}")
    print(f"📋 Déjà OCRisé (inchangé): {already_ocr_count}")
    print(f"❌ Erreurs: {error_count}")
    print(f"📂 Dossier: {PDF_DIR}")
    print("="*60)
    
else:
    print("ℹ️ OCR désactivé. Mettez USE_OCR = True pour l'activer.")
    print("   Les PDFs seront utilisés tels quels.")

## Étape 7 — Configurer Marker
Chargement des modèles Marker (~4GB) depuis le cache Drive.

**⚡ Premier lancement:** 10-20 minutes (téléchargement + sauvegarde sur Drive)

**🚀 Lancements suivants:** 1-2 minutes (chargement depuis Drive)

In [ ]:
import json
import re
import shutil
import base64
import logging
import warnings
import os
import sys
import time
from datetime import datetime
from pathlib import Path

# === OPTION: EXTRACTION D'IMAGES ===
EXTRACT_IMAGES = True  # ACTIVÉ pour extraire les images/figures

# === CONFIGURATION DES LOGS ===
# Désactiver les warnings non critiques
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

# Configurer le logging pour réduire la verbosité
logging.getLogger().setLevel(logging.WARNING)

# Réduire drastiquement les logs des bibliothèques tierces
for logger_name in ['transformers', 'torch', 'PIL', 'urllib3', 'filelock',
                     'huggingface_hub', 'marker', 'datasets']:
    logging.getLogger(logger_name).setLevel(logging.ERROR)

# Variables d'environnement pour réduire les logs
os.environ['TRANSFORMERS_VERBOSITY'] = 'error'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['HF_HUB_DISABLE_PROGRESS_BARS'] = '0'  # Garder les barres de progression

# S'assurer que le cache Drive est bien configuré
os.environ["HF_HOME"] = str(HF_CACHE_DRIVE)
os.environ["TRANSFORMERS_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["HUGGINGFACE_HUB_CACHE"] = str(HF_CACHE_DRIVE)
os.environ["TORCH_HOME"] = str(TORCH_CACHE_DRIVE)
# Forcer HF Transfer
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "1"

# === FORCER LE TÉLÉCHARGEMENT DES MODÈLES SURYA DEPUIS HUGGING FACE ===
os.environ["SURYA_LAYOUT_MODEL"] = "vikp/surya_layout"
os.environ["SURYA_REC_MODEL"] = "vikp/surya_rec"
os.environ["SURYA_DET_MODEL"] = "vikp/surya_det"
os.environ["SURYA_ORDER_MODEL"] = "vikp/surya_order"
os.environ["SURYA_TABLE_REC_MODEL"] = "vikp/surya_tablerec"
os.environ["SURYA_DOWNLOAD_BACKEND"] = "huggingface"

import torch
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
print("="*60)
print("🖥️ CONFIGURATION MATÉRIELLE")
print("="*60)
print(f"Device: {device}")
if device == "cuda":
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Vérifier le cache existant
def get_dir_size(path):
    total = 0
    if path.exists():
        for f in path.rglob("*"):
            if f.is_file():
                try:
                    total += f.stat().st_size
                except:
                    pass
    return total / 1e9

initial_cache_size = get_dir_size(DRIVE_CACHE_DIR)
print(f"\n💾 Cache Drive: {initial_cache_size:.2f} GB")

if initial_cache_size > 3:
    print("🚀 Modèles trouvés en cache! Chargement rapide...")
else:
    print("📥 Premier téléchargement des modèles Marker (~4GB)")
    print("   ⏱️ Durée estimée: 10-20 minutes")
    print("   💾 Les modèles seront sauvegardés sur Drive")

print("="*60)

# Chronomètre
start_time = time.time()

# Charger les modèles avec messages de progression
print("\n📦 Chargement des modèles Marker...")
if initial_cache_size > 3:
    print("   ⚡ Chargement depuis le cache Drive...")
else:
    print("   📥 Téléchargement en cours...")

from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.config.parser import ConfigParser

# Tenter d'importer le service LLM approprié
LLM_SERVICE_CLS = None
USE_OLLAMA = True

if USE_OLLAMA:
    try:
        from marker.llm.openai import OpenAILLMService
        LLM_SERVICE_CLS = OpenAILLMService
        print("✅ Service LLM: OpenAILLMService (compatible Ollama)")
    except ImportError:
        try:
            from marker.services.openai import OpenAILLMService
            LLM_SERVICE_CLS = OpenAILLMService
            print("✅ Service LLM: OpenAILLMService (via path alternatif)")
        except ImportError:
            print("⚠️ Impossible d'importer OpenAILLMService. Le LLM sera désactivé pour Marker.")
            USE_OLLAMA = False

# Configuration pour Ollama via API OpenAI
if USE_OLLAMA:
    os.environ["MARKER_LLM_PROVIDER"] = "openai"
    os.environ["OPENAI_BASE_URL"] = "http://localhost:11434/v1"
    os.environ["OPENAI_API_KEY"] = "ollama"
    os.environ["OPENAI_MODEL"] = "gemma3:4b"

# Configuration Marker OPTIMISÉE pour extraction complète
marker_config = {
    "workers": 2,
    "extract_images": EXTRACT_IMAGES,
    "images_as_base64": True,  # Récupérer les images en base64 pour éviter les erreurs de fichiers
    "use_llm": USE_OLLAMA,
    "llm_provider": "openai" if USE_OLLAMA else None,
    "llm_model": "gemma3:4b" if USE_OLLAMA else None,
    "force_ocr": False,
    "languages": ["fr", "en"],
    "paginate_output": True,
    "batch_size": 4 if device == "cuda" else 2,
    # Options supplémentaires pour améliorer l'extraction
    "output_format": "markdown",
}

print("\n⚙️ Création du dictionnaire de modèles...")
model_dict = create_model_dict()

print("✓ Configuration du convertisseur...")
config_parser = ConfigParser(marker_config)

converter = PdfConverter(
    config=config_parser.generate_config_dict(),
    artifact_dict=model_dict
)

# Résumé
elapsed = time.time() - start_time
final_cache_size = get_dir_size(DRIVE_CACHE_DIR)
downloaded = final_cache_size - initial_cache_size

print("\n" + "="*60)
print("✅ MARKER CONFIGURÉ AVEC SUCCÈS!")
print("="*60)
print(f"⏱️ Durée: {elapsed/60:.1f} minutes")
print(f"📦 Téléchargé cette session: {max(0, downloaded):.2f} GB")
print(f"💾 Cache total sur Drive: {final_cache_size:.2f} GB")
print(f"🖼️ Extraction d'images: {'Activée' if EXTRACT_IMAGES else 'Désactivée'}")
if USE_OLLAMA:
    print(f"🤖 LLM: Gemma 3 4B via Ollama (interface OpenAI)")
else:
    print("🤖 LLM: Désactivé")
print("="*60)
if downloaded > 0.5:
    print("\n💡 Les modèles sont maintenant en cache sur votre Drive.")

🖥️ CONFIGURATION MATÉRIELLE
Device: cuda
GPU: Tesla T4
VRAM: 15.8 GB

💾 Cache Drive: 3.53 GB
🚀 Modèles trouvés en cache! Chargement rapide...

📦 Chargement des modèles Marker...
   ⚡ Chargement depuis le cache Drive...
⚠️ Impossible d'importer OpenAILLMService. Le LLM sera désactivé pour Marker.

⚙️ Création du dictionnaire de modèles...
✓ Configuration du convertisseur...

✅ MARKER CONFIGURÉ AVEC SUCCÈS!
⏱️ Durée: 0.4 minutes
📦 Téléchargé cette session: 0.00 GB
💾 Cache total sur Drive: 3.53 GB
🖼️ Extraction d'images: Désactivée
🤖 LLM: Désactivé


## Étape 8 — Fonctions utilitaires
Extraction des références, figures et conversion PDF → Markdown.

In [ ]:
import gc
import fitz  # PyMuPDF pour compter les pages

def get_pdf_page_count(pdf_path):
    """Retourne le nombre de pages d'un PDF."""
    try:
        with fitz.open(str(pdf_path)) as doc:
            return len(doc)
    except:
        return 0

def clear_memory():
    """Libère la mémoire GPU et RAM."""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()

def create_converter_for_large_pdf(page_count):
    """Crée un convertisseur optimisé pour les gros PDFs."""
    # Ajuster les paramètres selon la taille
    if page_count > 300:
        batch = 1
        extract_img = False
        use_llm = False
    elif page_count > 200:
        batch = 2
        extract_img = False
        use_llm = False
    elif page_count > 100:
        batch = 2 if device == "cuda" else 1
        extract_img = EXTRACT_IMAGES
        use_llm = False
    else:
        return None  # Utiliser le convertisseur par défaut

    config = {
        "workers": 1,  # Réduire les workers
        "extract_images": extract_img,
        "images_as_base64": extract_img,
        "use_llm": use_llm,
        "force_ocr": False,
        "languages": ["fr", "en"],
        "paginate_output": True,
        "batch_size": batch,
        "output_format": "markdown",
    }

    config_parser = ConfigParser(config)
    return PdfConverter(
        config=config_parser.generate_config_dict(),
        artifact_dict=model_dict
    )


def extract_references_from_markdown(markdown_text):
    """Extrait la section références/bibliographie du Markdown avec patterns très améliorés."""
    references = {
        "references_text": "",
        "references_list": [],
        "reference_count": 0,
    }

    # Normaliser le texte (supprimer les sauts de ligne multiples excessifs)
    text = re.sub(r'\n{3,}', '\n\n', markdown_text)

    # === ÉTAPE 1: Trouver la section références ===
    ref_section = ""

    # Patterns pour trouver le DÉBUT de la section références
    section_start_patterns = [
        r"(?i)(?:^|\n)#{1,4}\s*(references?|références?|bibliography|bibliographie|works?\s*cited|cited\s*works?|literature|sources?|notes?\s*(?:and\s*)?references?)\s*\n",
        r"(?i)(?:^|\n)\*\*(references?|références?|bibliography|bibliographie)\*\*\s*\n",
        r"(?i)(?:^|\n)(REFERENCES?|RÉFÉRENCES?|BIBLIOGRAPHY|BIBLIOGRAPHIE|WORKS\s*CITED)\s*\n",
        r"(?i)(?:^|\n)_{2,}?\s*(references?|bibliography)\s*_{2,}?\s*\n",
    ]

    section_start_pos = -1
    for pattern in section_start_patterns:
        match = re.search(pattern, text)
        if match:
            section_start_pos = match.end()
            break

    if section_start_pos > 0:
        remaining = text[section_start_pos:]
        end_patterns = [
            r"\n#{1,3}\s+[A-Z]",
            r"\n\*\*[A-Z][a-z]+\*\*\s*\n",
            r"\n(?:APPENDIX|ANNEXE|ACKNOWLEDGMENT)",
        ]

        end_pos = len(remaining)
        for pattern in end_patterns:
            match = re.search(pattern, remaining, re.IGNORECASE)
            if match and match.start() < end_pos:
                end_pos = match.start()

        ref_section = remaining[:end_pos].strip()

    # === ÉTAPE 2: Si pas de section trouvée, chercher dans les derniers 30% ===
    if not ref_section or len(ref_section) < 100:
        last_portion = text[int(len(text) * 0.7):]
        ref_block_pattern = r"(?:^|\n)((?:\[\d+\]|\d+\.|[A-Z][a-z]+,?\s+[A-Z])[^\n]+(?:\n(?!\[\d+\]|\d+\.|[A-Z][a-z]+,\s+[A-Z])[^\n]+)*)"
        blocks = re.findall(ref_block_pattern, last_portion)

        if len(blocks) >= 5:
            ref_section = "\n".join(blocks)

    # === ÉTAPE 3: Extraction des références individuelles ===
    ref_lines = []

    if ref_section:
        references["references_text"] = ref_section[:5000]

        lines = ref_section.split('\n')
        current_ref = ""

        for line in lines:
            line = line.strip()
            if not line:
                if current_ref and len(current_ref) > 40:
                    ref_lines.append(current_ref)
                current_ref = ""
                continue

            is_new_ref = False
            if re.match(r"^\[\d+\]", line) or re.match(r"^\d+\.\s+[A-Z]", line):
                is_new_ref = True
            elif re.match(r"^[A-Z][a-z]+(?:[-'][A-Z][a-z]+)?,?\s+[A-Z]\..*\(\d{4}\)", line):
                is_new_ref = True
            elif re.match(r"^[A-Z][a-z]+,?\s+[A-Z]\.\s*(?:&|and|,)\s*[A-Z]", line):
                is_new_ref = True
            elif re.match(r"^[-•●○]\s+[A-Z]", line):
                is_new_ref = True
            elif re.match(r"^[A-Z][a-z]+\s+et\s+al\.", line):
                is_new_ref = True

            if is_new_ref:
                if current_ref and len(current_ref) > 40:
                    ref_lines.append(current_ref)
                current_ref = line
            elif current_ref:
                current_ref += " " + line
            elif len(line) > 50 and re.search(r'\d{4}', line):
                current_ref = line

        if current_ref and len(current_ref) > 40:
            ref_lines.append(current_ref)

    # === ÉTAPE 4: Extraction alternative ===
    if len(ref_lines) < 3:
        inline_patterns = [
            r"\[\d+\]\s+[A-Z][a-z]+(?:,?\s+[A-Z]\.)+[^.]+\.\s+[^.]+\.\s+\d{4}",
            r"[A-Z][a-z]+,?\s+[A-Z]\.(?:\s*[A-Z]\.)*\s*\(\d{4}\)\.\s+[^.]+\.[^.]+\.",
        ]

        for pattern in inline_patterns:
            matches = re.findall(pattern, text)
            for m in matches:
                if m not in ref_lines and len(m) > 50:
                    ref_lines.append(m.strip())

    # === ÉTAPE 5: Nettoyage et déduplication ===
    seen = set()
    unique_refs = []
    for ref in ref_lines:
        ref = re.sub(r'\s+', ' ', ref).strip()
        ref = re.sub(r'^[\[\d+\]|\d+\.|\-•●]\s*', '', ref).strip()
        key = ref[:80].lower()
        if key not in seen and len(ref) > 40:
            seen.add(key)
            unique_refs.append(ref)

    references["references_list"] = unique_refs[:200]
    references["reference_count"] = len(unique_refs)

    return references


def extract_figures_info(markdown_text, images_dict):
    """Extrait les informations sur les figures du document."""
    figures = []
    seen_labels = set()

    fig_patterns = [
        r"(?i)(?:^|\n)\s*((?:figure|fig\.?)\s*(\d+(?:\.\d+)?)[:\s.]*([^\n]{0,200}))",
        r"(?i)(?:^|\n)\s*((?:figure|fig\.?)\s*(\d+(?:\.\d+)?)\s*[—–-]\s*([^\n]{0,200}))",
        r"(?i)(?:^|\n)\s*((?:table(?:au)?)\s*(\d+(?:\.\d+)?)[:\s.]*([^\n]{0,200}))",
        r"(?i)(?:^|\n)\s*((?:chart|graph|diagram|graphique|schéma)\s*(\d+(?:\.\d+)?)?[:\s.]*([^\n]{0,150}))",
    ]

    for pattern in fig_patterns:
        for match in re.finditer(pattern, markdown_text, re.MULTILINE):
            groups = match.groups()
            if len(groups) >= 1:
                full_match = groups[0] if groups[0] else match.group(0)
                fig_num = groups[1] if len(groups) > 1 and groups[1] else ""
                title = groups[2].strip() if len(groups) > 2 and groups[2] else ""
                title = re.sub(r'^[:\s.—–-]+', '', title).strip()
                label = full_match.strip()[:100]
                if label.lower() not in seen_labels:
                    seen_labels.add(label.lower())
                    figures.append({
                        "label": label,
                        "number": fig_num,
                        "title": title,
                        "type": "figure",
                    })

    img_md_pattern = r"!\[([^\]]*)\]\(([^)]+)\)"
    for match in re.finditer(img_md_pattern, markdown_text):
        alt_text = match.group(1).strip()
        img_src = match.group(2).strip()
        if alt_text and alt_text.lower() not in seen_labels:
            seen_labels.add(alt_text.lower())
            figures.append({
                "label": alt_text or f"Image: {img_src[:50]}",
                "title": alt_text,
                "path": img_src,
                "type": "embedded_image",
            })

    if images_dict:
        for idx, img_name in enumerate(images_dict.keys(), 1):
            img_label = str(img_name)
            if img_label.lower() not in seen_labels:
                seen_labels.add(img_label.lower())
                figures.append({
                    "label": img_label,
                    "title": "",
                    "path": img_label,
                    "type": "extracted_image",
                    "index": idx,
                })

    return figures


def save_figures(images_dict, doc_folders):
    """Sauvegarde les figures extraites."""
    if not images_dict:
        return []

    figures_folder = doc_folders["figures"]
    saved_paths = []
    used_names = set()

    for idx, (img_name, img_data) in enumerate(images_dict.items(), 1):
        safe_name = re.sub(r"[^a-zA-Z0-9_-]+", "_", str(img_name))
        safe_name = re.sub(r"_+", "_", safe_name).strip("_")[:100]
        if not safe_name or len(safe_name) < 3:
            safe_name = f"figure_{idx:03d}"

        base_name = safe_name
        counter = 1
        while safe_name in used_names:
            safe_name = f"{base_name}_{counter}"
            counter += 1
        used_names.add(safe_name)

        img_path = figures_folder / f"{safe_name}.png"

        try:
            if isinstance(img_data, Image.Image):
                if img_data.mode in ('RGBA', 'LA', 'P'):
                    rgb_image = Image.new('RGB', img_data.size, (255, 255, 255))
                    if img_data.mode == 'P':
                        img_data = img_data.convert('RGBA')
                    rgb_image.paste(img_data, mask=img_data.split()[-1] if img_data.mode in ('RGBA', 'LA') else None)
                    img_data = rgb_image
                img_data.save(img_path, "PNG", optimize=True)
            elif isinstance(img_data, (bytes, bytearray)):
                with open(img_path, "wb") as f:
                    f.write(img_data)
            elif isinstance(img_data, str):
                if img_data.startswith("data:image"):
                    b64_data = img_data.split(",", 1)[1] if "," in img_data else img_data
                    with open(img_path, "wb") as f:
                        f.write(base64.b64decode(b64_data))
                elif Path(img_data).exists():
                    shutil.copy(img_data, img_path)
                else:
                    try:
                        with open(img_path, "wb") as f:
                            f.write(base64.b64decode(img_data))
                    except:
                        continue
            else:
                continue
            saved_paths.append(str(img_path))
        except Exception as e:
            continue

    return saved_paths


def extract_metadata_from_markdown(markdown_text):
    """Extrait les métadonnées du document."""
    metadata = {"title": "", "authors": [], "abstract": "", "keywords": [], "date": ""}

    title_patterns = [
        r"^#\s+([^\n]+)",
        r"^\*\*([^\*\n]{10,150})\*\*",
        r"^([A-Z][^\n]{20,150})\n[=]+",
    ]
    for pattern in title_patterns:
        match = re.search(pattern, markdown_text[:2000], re.MULTILINE)
        if match:
            metadata["title"] = match.group(1).strip()
            break

    abstract_pattern = r"(?i)(?:abstract|résumé|summary)[:\s]*\n?([\s\S]{50,1500}?)(?=\n\n|\n#{1,3}|\n\*\*[A-Z])"
    match = re.search(abstract_pattern, markdown_text[:5000])
    if match:
        metadata["abstract"] = match.group(1).strip()

    keywords_pattern = r"(?i)(?:keywords?|mots[- ]?clés?)[:\s]*([^\n]+)"
    match = re.search(keywords_pattern, markdown_text[:5000])
    if match:
        kw_text = match.group(1)
        keywords = re.split(r'[,;•·]', kw_text)
        metadata["keywords"] = [k.strip() for k in keywords if k.strip() and len(k.strip()) > 2]

    dates = re.findall(r"\b((?:19|20)\d{2})\b", markdown_text[:3000])
    if dates:
        metadata["date"] = dates[0]

    return metadata


def convert_pdf_complete(pdf_path, doc_name):
    """Conversion complète d'un PDF avec gestion mémoire optimisée."""
    doc_folders = get_doc_folders(doc_name)

    result_data = {
        "doc_name": doc_name,
        "doc_folder": str(doc_folders["root"]),
        "markdown_path": "",
        "figures": [],
        "figures_paths": [],
        "references": {},
        "metadata": {},
        "page_count": 0,
        "error": None,
    }

    try:
        # Compter les pages pour adapter le traitement
        page_count = get_pdf_page_count(pdf_path)
        result_data["page_count"] = page_count

        # Choisir le convertisseur adapté
        if page_count > 100:
            print(f"   ⚠️ Document volumineux ({page_count} pages) - Mode optimisé mémoire")
            current_converter = create_converter_for_large_pdf(page_count)
            if current_converter is None:
                current_converter = converter
        else:
            current_converter = converter

        print(f"   📄 Conversion de {pdf_path.name} ({page_count} pages)...")

        # Libérer la mémoire avant conversion
        clear_memory()

        try:
            result = current_converter(str(pdf_path))
        except Exception as conv_error:
            error_msg = str(conv_error).lower()
            if "extension" in error_msg or "image" in error_msg or "unknown" in error_msg or "memory" in error_msg:
                print(f"   ⚠️ Erreur, nouvelle tentative en mode minimal...")
                clear_memory()

                temp_config = {
                    "workers": 1,
                    "extract_images": False,
                    "images_as_base64": False,
                    "use_llm": False,
                    "force_ocr": False,
                    "languages": ["fr", "en"],
                    "paginate_output": True,
                    "batch_size": 1,
                }
                config_parser = ConfigParser(temp_config)
                temp_converter = PdfConverter(
                    config=config_parser.generate_config_dict(),
                    artifact_dict=model_dict,
                    llm_service=None
                )
                result = temp_converter(str(pdf_path))
                del temp_converter
                clear_memory()
            else:
                raise

        markdown_text = getattr(result, "markdown", "") or ""
        images_dict = getattr(result, "images", {}) or {}

        print(f"   📊 Markdown: {len(markdown_text)} caractères")
        print(f"   🖼️ Images brutes de Marker: {len(images_dict)}")

        # Nettoyer les images
        cleaned_images = {}
        if images_dict:
            for key, value in images_dict.items():
                if key and value is not None:
                    if not any(char in str(key) for char in ['?', '*', '<', '>', '|', '\x00']):
                        cleaned_images[key] = value

        images_dict = cleaned_images

        # Sauvegarder le Markdown
        md_path = doc_folders["root"] / f"{doc_name}.md"
        with open(md_path, "w", encoding="utf-8") as f:
            f.write(markdown_text)

        result_data["markdown_path"] = str(md_path)

        # Extractions
        result_data["figures_paths"] = save_figures(images_dict, doc_folders)
        result_data["figures"] = extract_figures_info(markdown_text, images_dict)
        result_data["references"] = extract_references_from_markdown(markdown_text)
        result_data["metadata"] = extract_metadata_from_markdown(markdown_text)

        # Libérer les variables lourdes
        del markdown_text, images_dict, cleaned_images, result
        clear_memory()

        # Sauvegarder les résultats
        if result_data["references"].get("reference_count", 0) > 0:
            ref_path = doc_folders["references"] / f"{doc_name}_references.json"
            with open(ref_path, "w", encoding="utf-8") as f:
                json.dump(result_data["references"], f, ensure_ascii=False, indent=2)

        if result_data["metadata"].get("title"):
            meta_path = doc_folders["analyses"] / f"{doc_name}_metadata.json"
            with open(meta_path, "w", encoding="utf-8") as f:
                json.dump(result_data["metadata"], f, ensure_ascii=False, indent=2)

        return result_data

    except Exception as e:
        import traceback
        result_data["error"] = str(e)
        error_log = doc_folders["logs"] / "error.txt"
        with open(error_log, "w", encoding="utf-8") as f:
            f.write(f"Erreur: {str(e)}\nTraceback:\n{traceback.format_exc()}\nDate: {datetime.now().isoformat()}")
        clear_memory()
        return result_data


print("✅ Fonctions utilitaires optimisées définies.")
print("   • Gestion mémoire: libération automatique GPU/RAM")
print("   • Gros fichiers (>100 pages): mode optimisé automatique")
print("   • Extraction de références: patterns académiques améliorés")

✅ Fonctions utilitaires définies. 


## Étape 9 — LangExtract
Extraction structurée avec LangExtract (activé par défaut avec Ollama).

In [ ]:
USE_LANGEXTRACT = True

# Vérifier si Ollama est disponible (indépendamment de la config Marker)
OLLAMA_AVAILABLE = False
try:
    import requests
    response = requests.get("http://localhost:11434/api/tags", timeout=2)
    if response.status_code == 200:
        OLLAMA_AVAILABLE = True
        print("✅ Ollama détecté - Utilisation de Gemma 3 4B")
except:
    print("⚠️ Ollama non disponible - LangExtract utilisera l'API directe")

LANGEXTRACT_CONFIG = {
    "provider": "ollama" if OLLAMA_AVAILABLE else "openai",
    "model": "gemma3:4b" if OLLAMA_AVAILABLE else "gpt-3.5-turbo",
    "base_url": "http://localhost:11434" if OLLAMA_AVAILABLE else None,
}

# Prompt pour l'extraction structurée (utilisé avec Ollama directement si LangExtract échoue)
EXTRACTION_PROMPT = """Analysez ce document académique et retournez un JSON structuré avec:
{
  "contexte": {"theme": "", "zone_geographique": "", "periode": ""},
  "acteurs": {"institutions": [], "pays": [], "organisations": []},
  "concepts_cles": [],
  "donnees_chiffrees": [],
  "references_principales": []
}
Répondez UNIQUEMENT avec le JSON, sans texte avant ou après."""

def _safe_json(obj):
    try:
        return json.loads(json.dumps(obj))
    except Exception:
        return {"raw": str(obj)}


def extract_with_langextract(markdown_text, doc_name, doc_folders, references_data=None, figures_data=None):
    """Extraction structurée avec LangExtract ou Ollama directement."""
    if not USE_LANGEXTRACT:
        return {"status": "skipped", "reason": "USE_LANGEXTRACT=False"}

    # Préparer le texte enrichi (limiter la taille pour le LLM)
    max_chars = 15000  # Limite pour éviter les timeouts
    enriched_text = markdown_text[:max_chars]

    if references_data and references_data.get("reference_count", 0) > 0:
        enriched_text += f"\n\n[RÉFÉRENCES EXTRAITES: {references_data['reference_count']}]"

    if figures_data:
        enriched_text += f"\n\n[FIGURES DÉTECTÉES: {len(figures_data)}]"

    result = None

    # === TENTATIVE 1: LangExtract (nouvelle API) ===
    try:
        import langextract as lx

        # Essayer différentes signatures d'API
        if hasattr(lx, "extract"):
            # Nouvelle API sans prompt
            try:
                extraction = lx.extract(enriched_text)
                result = _safe_json(extraction)
            except TypeError:
                # Essayer avec d'autres paramètres
                try:
                    extraction = lx.extract(enriched_text, model=LANGEXTRACT_CONFIG["model"])
                    result = _safe_json(extraction)
                except:
                    pass
        elif hasattr(lx, "LangExtract"):
            try:
                extractor = lx.LangExtract()
                extraction = extractor.extract(enriched_text)
                result = _safe_json(extraction)
            except:
                pass
    except Exception as e:
        pass  # Continuer avec Ollama

    # === TENTATIVE 2: Ollama directement ===
    if result is None and OLLAMA_AVAILABLE:
        try:
            import requests

            ollama_payload = {
                "model": "gemma3:4b",
                "prompt": f"{EXTRACTION_PROMPT}\n\nDOCUMENT:\n{enriched_text[:10000]}",
                "stream": False,
                "options": {"temperature": 0.1}
            }

            resp = requests.post(
                "http://localhost:11434/api/generate",
                json=ollama_payload,
                timeout=120
            )

            if resp.status_code == 200:
                ollama_result = resp.json()
                response_text = ollama_result.get("response", "")

                # Extraire le JSON de la réponse
                json_match = re.search(r'\{[\s\S]*\}', response_text)
                if json_match:
                    try:
                        result = json.loads(json_match.group())
                    except json.JSONDecodeError:
                        result = {"raw_response": response_text[:500]}
                else:
                    result = {"raw_response": response_text[:500]}
        except Exception as e:
            result = {"status": "error", "error": f"Ollama: {str(e)}"}

    # === FALLBACK: Extraction basique sans LLM ===
    if result is None:
        result = {
            "status": "fallback",
            "note": "Extraction LLM indisponible, données de base uniquement",
            "references_count": references_data.get("reference_count", 0) if references_data else 0,
            "figures_count": len(figures_data) if figures_data else 0,
        }

    # Sauvegarder le résultat
    extraction_path = doc_folders["analyses"] / f"{doc_name}_langextract.json"
    with open(extraction_path, "w", encoding="utf-8") as f:
        json.dump(result, f, ensure_ascii=False, indent=2)

    return result


print(f"✅ LangExtract configuré")
print(f"   Provider: {LANGEXTRACT_CONFIG['provider']}")
print(f"   Model: {LANGEXTRACT_CONFIG['model']}")
print(f"   Fallback Ollama: {'Activé' if OLLAMA_AVAILABLE else 'Désactivé'}")

✅ Ollama détecté - Utilisation de Gemma 3 4B
✅ LangExtract configuré (Provider: ollama, Model: gemma3:4b)


## Étape 10 — Test sur un PDF (optionnel)
Permet de valider la configuration avant le batch.

In [ ]:
pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])

if pdf_files:
    sample_path = pdf_files[0]
    sample_name = sample_path.stem
    print(f"🧪 Test sur: {sample_path.name}")
    print("⏳ Conversion en cours...\n")

    test_start = time.time()
    sample_result = convert_pdf_complete(sample_path, sample_name)
    test_elapsed = time.time() - test_start

    if sample_result.get("error"):
        print(f"\n❌ Erreur: {sample_result['error']}")
    else:
        print(f"\n" + "="*60)
        print("✅ CONVERSION RÉUSSIE!")
        print("="*60)
        print(f"⏱️ Durée: {test_elapsed:.1f}s")
        print(f"\n📂 Dossier de sortie:")
        print(f"   {sample_result['doc_folder']}")

        print(f"\n📄 MARKDOWN:")
        print(f"   Fichier: {sample_result['markdown_path']}")
        # Afficher un extrait du contenu
        try:
            with open(sample_result['markdown_path'], 'r', encoding='utf-8') as f:
                content = f.read()
            print(f"   Taille: {len(content):,} caractères")
            print(f"   Lignes: {content.count(chr(10)):,}")
        except:
            pass

        print(f"\n🖼️ FIGURES:")
        figures = sample_result.get('figures', [])
        print(f"   Détectées dans le texte: {len(figures)}")
        print(f"   Images sauvegardées: {len(sample_result.get('figures_paths', []))}")
        if figures:
            print("   Exemples:")
            for fig in figures[:5]:
                fig_type = fig.get('type', 'unknown')
                label = fig.get('label', '')[:60]
                print(f"      [{fig_type}] {label}")
            if len(figures) > 5:
                print(f"      ... et {len(figures)-5} autres")

        print(f"\n📚 RÉFÉRENCES:")
        refs = sample_result.get('references', {})
        ref_count = refs.get('reference_count', 0)
        print(f"   Nombre: {ref_count}")
        if ref_count > 0:
            print("   Exemples:")
            for ref in refs.get('references_list', [])[:3]:
                print(f"      • {ref[:80]}...")
            if ref_count > 3:
                print(f"      ... et {ref_count-3} autres")
        elif refs.get('references_text'):
            print(f"   ⚠️ Section trouvée mais parsing échoué")
            print(f"   Texte brut ({len(refs['references_text'])} chars):")
            print(f"      {refs['references_text'][:200]}...")

        print(f"\n📋 MÉTADONNÉES:")
        meta = sample_result.get('metadata', {})
        if meta.get('title'):
            print(f"   Titre: {meta['title'][:80]}")
        if meta.get('abstract'):
            print(f"   Abstract: {meta['abstract'][:100]}...")
        if meta.get('keywords'):
            print(f"   Keywords: {', '.join(meta['keywords'][:5])}")
        if meta.get('date'):
            print(f"   Date: {meta['date']}")

        print("\n" + "="*60)

        # === DIAGNOSTIC AVANCÉ si pas de figures/refs ===
        if len(figures) == 0 or ref_count == 0:
            print("\n🔍 DIAGNOSTIC (extraction insuffisante):")
            try:
                with open(sample_result['markdown_path'], 'r', encoding='utf-8') as f:
                    md_content = f.read()

                # Chercher des patterns de figures
                fig_matches = re.findall(r"(?i)(figure|fig\.?|table|tableau)\s*\d", md_content)
                print(f"   Mentions 'figure/table' trouvées: {len(fig_matches)}")
                if fig_matches:
                    print(f"      Exemples: {fig_matches[:5]}")

                # Chercher des images markdown
                img_matches = re.findall(r"!\[[^\]]*\]\([^)]+\)", md_content)
                print(f"   Images markdown (![]()): {len(img_matches)}")

                # Chercher des sections de références
                ref_sections = re.findall(r"(?i)(references?|bibliography|bibliographie)", md_content)
                print(f"   Mentions 'references/bibliography': {len(ref_sections)}")

                # Chercher des citations numérotées
                citations = re.findall(r"\[\d+\]", md_content)
                print(f"   Citations numériques [n]: {len(citations)}")

                # Afficher la fin du document (souvent les références)
                print(f"\n   📜 Fin du document (2000 derniers chars):")
                print("   " + "-"*50)
                print(md_content[-2000:].replace('\n', '\n   '))
            except Exception as e:
                print(f"   Erreur diagnostic: {e}")

else:
    print("❌ Aucun PDF trouvé dans le dossier d'entrée.")
    print(f"   Vérifiez le chemin: {INPUT_DIR}")

🧪 Test sur:  Against Sovereignty in Cyberspace.pdf
⏳ Conversion en cours...

   DEBUG: Tentative de conversion initiale pour Against Sovereignty in Cyberspace.pdf...


Recognizing tables: 100%|██████████| 1/1 [00:00<00:00,  3.93it/s]
Detecting bboxes: 0it [00:00, ?it/s]


   DEBUG: Première conversion réussie.
✅ Conversion réussie en 104.1s! 
   📂 Dossier: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD/Against Sovereignty in Cyberspace
   📄 Markdown: /content/drive/MyDrive/Géopolitique et Souveraineté Numériques/ALL/ALLMD/Against Sovereignty in Cyberspace/Against Sovereignty in Cyberspace.md
   🖼️ Figures: 0
   📚 Références: 0


## Étape 11 — Pipeline complet avec reprise
Traitement batch + logs + reprise automatique.

In [ ]:
import time as time_module

# === CONFIGURATION GROS FICHIERS ===
MAX_PAGES_BEFORE_WARNING = 100
MAX_PAGES_BEFORE_SKIP = 500  # Fichiers > 500 pages seront traités en dernier
MEMORY_CHECK_INTERVAL = 5   # Vérifier la mémoire tous les N fichiers

def get_memory_usage():
    """Retourne l'utilisation mémoire en GB."""
    try:
        import psutil
        process = psutil.Process()
        return process.memory_info().rss / 1e9
    except:
        return 0

def process_all_documents():
    """Traite tous les documents PDF avec reprise automatique et gestion mémoire."""

    all_pdfs = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])

    if not all_pdfs:
        print("❌ Aucun PDF trouvé dans le dossier d'entrée!")
        print(f"   Chemin vérifié: {INPUT_DIR}")
        return [], []

    # Trier les PDFs par taille (plus petits d'abord pour éviter les crashs)
    print("📊 Analyse des fichiers...")
    pdf_info = []
    for pdf in all_pdfs:
        page_count = get_pdf_page_count(pdf)
        pdf_info.append({"path": pdf, "pages": page_count})

    # Trier: petits fichiers d'abord, très gros en dernier
    pdf_info.sort(key=lambda x: (x["pages"] > MAX_PAGES_BEFORE_SKIP, x["pages"]))

    log_file = GLOBAL_LOGS_DIR / f"processing_log_{datetime.now().strftime('%Y%m%d_%H%M%S')}.txt"
    progress_file = GLOBAL_LOGS_DIR / "progress.json"

    def log_message(message):
        print(message)
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(message + "\n")

    processed_files = set()
    if progress_file.exists():
        try:
            with open(progress_file, "r", encoding="utf-8") as f:
                progress_data = json.load(f)
                processed_files = set(progress_data.get("processed", []))
            log_message(f"📂 Reprise: {len(processed_files)} fichiers déjà traités")
        except Exception:
            processed_files = set()

    remaining_pdfs = [p for p in pdf_info if p["path"].name not in processed_files]

    # Statistiques
    small_files = sum(1 for p in remaining_pdfs if p["pages"] <= MAX_PAGES_BEFORE_WARNING)
    medium_files = sum(1 for p in remaining_pdfs if MAX_PAGES_BEFORE_WARNING < p["pages"] <= MAX_PAGES_BEFORE_SKIP)
    large_files = sum(1 for p in remaining_pdfs if p["pages"] > MAX_PAGES_BEFORE_SKIP)

    log_message("="*60)
    log_message(f"📄 Total PDFs: {len(all_pdfs)}")
    log_message(f"✅ Déjà traités: {len(processed_files)}")
    log_message(f"⏳ Restants: {len(remaining_pdfs)}")
    log_message(f"   • Petits (<{MAX_PAGES_BEFORE_WARNING} pages): {small_files}")
    log_message(f"   • Moyens ({MAX_PAGES_BEFORE_WARNING}-{MAX_PAGES_BEFORE_SKIP} pages): {medium_files}")
    log_message(f"   • Gros (>{MAX_PAGES_BEFORE_SKIP} pages): {large_files}")
    log_message("="*60)

    if not remaining_pdfs:
        log_message("✅ Tous les fichiers ont déjà été traités!")
        return [], []

    results = []
    errors = []
    start_time = time_module.time()

    def save_progress():
        with open(progress_file, "w", encoding="utf-8") as f:
            json.dump({
                "processed": list(processed_files),
                "last_update": datetime.now().isoformat()
            }, f, ensure_ascii=False, indent=2)

    for idx, pdf_data in enumerate(remaining_pdfs, 1):
        pdf_path = pdf_data["path"]
        page_count = pdf_data["pages"]
        doc_name = pdf_path.stem
        doc_start_time = time_module.time()

        # Vérification mémoire périodique
        if idx % MEMORY_CHECK_INTERVAL == 0:
            mem_usage = get_memory_usage()
            if mem_usage > 10:  # > 10 GB utilisés
                log_message(f"   ⚠️ Mémoire élevée ({mem_usage:.1f} GB) - Libération...")
                clear_memory()

        log_message(f"\n[{idx}/{len(remaining_pdfs)}] 🚀 Traitement: {pdf_path.name}")
        log_message(f"   📄 Pages: {page_count}")

        if page_count > MAX_PAGES_BEFORE_SKIP:
            log_message(f"   ⚠️ Document très volumineux - Traitement en mode ultra-léger")

        conversion_result = convert_pdf_complete(pdf_path, doc_name)

        if conversion_result.get("error"):
            errors.append({"file": pdf_path.name, "error": conversion_result["error"], "pages": page_count})
            log_message(f"   ❌ Erreur conversion: {conversion_result['error'][:100]}")
            processed_files.add(pdf_path.name)
            save_progress()
            clear_memory()
            continue

        doc_folders = get_doc_folders(doc_name)

        log_message(
            f"   🖼️ Figures: {len(conversion_result['figures'])} trouvées, "
            f"{len(conversion_result['figures_paths'])} sauvegardées"
        )
        log_message(
            f"   📚 Références: {conversion_result['references'].get('reference_count', 0)} extraites"
        )

        extraction = None
        # Désactiver LangExtract pour les très gros fichiers (économie mémoire)
        if USE_LANGEXTRACT and page_count <= MAX_PAGES_BEFORE_SKIP:
            log_message("   🔍 Extraction LangExtract...")
            try:
                with open(conversion_result['markdown_path'], 'r', encoding='utf-8') as f:
                    md_content = f.read()
                extraction = extract_with_langextract(
                    md_content,
                    doc_name,
                    doc_folders,
                    conversion_result['references'],
                    conversion_result['figures'],
                )
                del md_content
                if extraction.get("status") == "error":
                    log_message(f"   ⚠️ LangExtract: {extraction.get('error', 'Erreur inconnue')[:50]}")
                else:
                    log_message("   ✅ LangExtract terminé")
            except Exception as e:
                log_message(f"   ⚠️ LangExtract ignoré: {str(e)[:50]}")
        elif page_count > MAX_PAGES_BEFORE_SKIP:
            log_message("   ⏭️ LangExtract ignoré (document trop volumineux)")

        analysis = {
            "doc_name": doc_name,
            "source_pdf": str(pdf_path),
            "doc_folder": str(doc_folders["root"]),
            "processed_at": datetime.now().isoformat(),
            "processing_time_seconds": round(time_module.time() - doc_start_time, 2),
            "page_count": page_count,
            "conversion": {
                "markdown_path": conversion_result['markdown_path'],
                "figures_count": len(conversion_result['figures']),
                "figures_saved": len(conversion_result['figures_paths']),
                "figures_paths": conversion_result['figures_paths'],
                "references_count": conversion_result['references'].get('reference_count', 0),
            },
            "references": conversion_result['references'],
            "figures": conversion_result['figures'],
            "langextract": extraction,
        }

        analysis_path = doc_folders["analyses"] / f"{doc_name}_analysis.json"
        with open(analysis_path, "w", encoding="utf-8") as f:
            json.dump(analysis, f, ensure_ascii=False, indent=2)

        doc_log_path = doc_folders["logs"] / "processing.log"
        with open(doc_log_path, "w", encoding="utf-8") as f:
            f.write(f"Traité le: {datetime.now().isoformat()}\n")
            f.write(f"Pages: {page_count}\n")
            f.write(f"Durée: {analysis['processing_time_seconds']} secondes\n")
            f.write(f"Figures: {len(conversion_result['figures_paths'])}\n")
            f.write(f"Références: {conversion_result['references'].get('reference_count', 0)}\n")

        results.append(analysis)
        processed_files.add(pdf_path.name)
        save_progress()

        elapsed = time_module.time() - doc_start_time
        log_message(f"   ⏱️ Durée: {elapsed:.1f}s")

        # Libérer la mémoire après chaque document
        del conversion_result, analysis
        clear_memory()

    total_time = time_module.time() - start_time
    log_message("\n" + "="*60)
    log_message(f"✅ Traitement terminé en {total_time/60:.1f} minutes")
    log_message(f"   📄 Réussis: {len(results)}")
    log_message(f"   ❌ Erreurs: {len(errors)}")
    log_message("="*60)

    return results, errors


def export_all_results(results):
    """Exporte les résultats globaux."""
    if not results:
        print("ℹ️ Aucun résultat à exporter.")
        return

    summary_path = GLOBAL_LOGS_DIR / "_SUMMARY.json"
    report_md_path = GLOBAL_LOGS_DIR / "_REPORT.md"

    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=2)

    all_references = []
    for data in results:
        refs = data.get("references", {})
        if refs.get("references_list"):
            for ref in refs["references_list"]:
                all_references.append({
                    "document": data.get("doc_name", ""),
                    "reference": ref,
                })

    if all_references:
        biblio_path = GLOBAL_LOGS_DIR / "_BIBLIOGRAPHIE_COMPLETE.json"
        with open(biblio_path, "w", encoding="utf-8") as f:
            json.dump(all_references, f, ensure_ascii=False, indent=2)
        print(f"📚 Bibliographie: {biblio_path} ({len(all_references)} références)")

    all_figures = []
    for data in results:
        for fig_path in data.get("conversion", {}).get("figures_paths", []):
            all_figures.append({
                "document": data.get("doc_name", ""),
                "path": fig_path,
            })

    if all_figures:
        figures_index_path = GLOBAL_LOGS_DIR / "_INDEX_FIGURES.json"
        with open(figures_index_path, "w", encoding="utf-8") as f:
            json.dump(all_figures, f, ensure_ascii=False, indent=2)
        print(f"🖼️ Index figures: {figures_index_path} ({len(all_figures)} figures)")

    # Statistiques par taille
    small = [r for r in results if r.get("page_count", 0) <= 100]
    medium = [r for r in results if 100 < r.get("page_count", 0) <= 500]
    large = [r for r in results if r.get("page_count", 0) > 500]

    md_lines = []
    md_lines.append("# Rapport de traitement MFEGSN\n\n")
    md_lines.append(f"**Date:** {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")
    md_lines.append("## Statistiques globales\n\n")
    md_lines.append(f"- **Total documents traités:** {len(results)}\n")
    md_lines.append(f"- **Total figures extraites:** {len(all_figures)}\n")
    md_lines.append(f"- **Total références:** {len(all_references)}\n\n")
    md_lines.append("### Par taille de document\n\n")
    md_lines.append(f"- Petits (≤100 pages): {len(small)}\n")
    md_lines.append(f"- Moyens (101-500 pages): {len(medium)}\n")
    md_lines.append(f"- Gros (>500 pages): {len(large)}\n\n")
    md_lines.append("## Documents traités\n\n")

    for data in results:
        conv = data.get("conversion", {})
        pages = data.get("page_count", 0)
        md_lines.append(f"### {data.get('doc_name', '')}\n\n")
        md_lines.append(f"- **Pages:** {pages}\n")
        md_lines.append(f"- **Figures:** {conv.get('figures_count', 0)}\n")
        md_lines.append(f"- **Références:** {conv.get('references_count', 0)}\n")
        md_lines.append(f"- **Durée:** {data.get('processing_time_seconds', 0)}s\n\n")

    with open(report_md_path, "w", encoding="utf-8") as f:
        f.write("".join(md_lines))

    print(f"✅ Résultats exportés:")
    print(f"   📊 Summary: {summary_path}")
    print(f"   📝 Report: {report_md_path}")


print("✅ Fonctions de pipeline définies.")
print("   • Tri intelligent: petits fichiers traités en premier")
print("   • Libération mémoire automatique")
print("   • Gros fichiers (>500 pages): mode ultra-léger")

## Étape 12 — Lancer le traitement
Exécutez cette cellule pour lancer le traitement complet.

In [ ]:
# Recharger la liste des PDFs depuis INPUT_DIR pour s'assurer qu'elle est à jour
print("📂 Vérification du dossier d'entrée...")
print(f"   Chemin: {INPUT_DIR}")

# Recharger la liste des fichiers
pdf_files = sorted([p for p in INPUT_DIR.iterdir() if p.suffix.lower() == ".pdf"])
print(f"   PDFs trouvés: {len(pdf_files)}")

if len(pdf_files) == 0:
    print("\n⚠️ ATTENTION: Aucun PDF trouvé!")
    print("   Vérifiez que:")
    print("   1. Le chemin INPUT_DIR est correct")
    print("   2. Des fichiers PDF sont présents dans ce dossier")
    print("   3. Google Drive est bien monté")
else:
    for i, pdf in enumerate(pdf_files[:5], 1):
        print(f"   {i}. {pdf.name}")
    if len(pdf_files) > 5:
        print(f"   ... et {len(pdf_files) - 5} autres")

print("\n" + "="*60)
print("🚀 Démarrage du pipeline...")
print("="*60)

results, errors = process_all_documents()

if results:
    export_all_results(results)

print("\n" + "="*60)
print("✅ PIPELINE TERMINÉ")
print("="*60)

if errors:
    print(f"\n⚠️ {len(errors)} erreur(s) rencontrée(s):")
    for err in errors[:5]:
        print(f"   - {err['file']}: {err['error'][:50]}...")
    if len(errors) > 5:
        print(f"   ... et {len(errors) - 5} autres erreurs")

## Utilitaires — Gestion du cache
Cellules optionnelles pour gérer le cache des modèles sur Drive.

In [ ]:
# === AFFICHER LA TAILLE DU CACHE ===
def show_cache_info():
    """Affiche les informations sur le cache Drive."""
    print("="*60)
    print("💾 INFORMATIONS CACHE")
    print("="*60)

    for name, path in [("HuggingFace", HF_CACHE_DRIVE),
                       ("Torch", TORCH_CACHE_DRIVE),
                       ("Datalab", DATALAB_CACHE_DRIVE)]:
        size = get_dir_size(path)
        print(f"📂 {name}: {size:.2f} GB")

    total = get_dir_size(DRIVE_CACHE_DIR)
    print(f"\n📦 Total: {total:.2f} GB")
    print(f"📍 Emplacement: {DRIVE_CACHE_DIR}")
    print("="*60)

# Appeler la fonction pour afficher les infos
show_cache_info()

In [ ]:
# === SUPPRIMER LE CACHE (si nécessaire) ===
# ⚠️ ATTENTION:  Cela supprimera tous les modèles en cache!
# Décommentez les lignes ci-dessous pour exécuter.

# import shutil
#
# if DRIVE_CACHE_DIR. exists():
#     print(f"⚠️ Suppression du cache:  {DRIVE_CACHE_DIR}")
#     shutil.rmtree(DRIVE_CACHE_DIR)
#     print("✅ Cache supprimé.  Les modèles seront retéléchargés au prochain lancement.")
# else:
#     print("ℹ️ Aucun cache à supprimer.")

print("ℹ️ Décommentez le code ci-dessus pour supprimer le cache.")